# Evaluate MobileNet V2

Install Weights and Biases since it is not in Google Colab by default

In [1]:
!pip install wandb -qqq

     |████████████████████████████████| 1.8MB 8.7MB/s 
     |████████████████████████████████| 163kB 29.4MB/s 
     |████████████████████████████████| 102kB 11.5MB/s 
     |████████████████████████████████| 133kB 23.8MB/s 
     |████████████████████████████████| 102kB 11.3MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 


Import relevant libraries

In [2]:
from google.colab import drive
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

import pandas as pd
import tensorflow as tf
import wandb

Mount Google Drive to access images

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
WANDB_API_KEY="aa601783a9e0b5f4bde1480428fb5318ad3dc0f2"

In [5]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
# Initialise Weights and Biases API
# Hyperparameters

# previous config:
# config={
#     'batch_size': 128,  # tried 64, 256
#     'epochs': 10,  # tried 20
#     'seed': 0,  # tried 42
#     'learning_rate': 0.0001,  # tried 0.001, 0.00005
#     'dropout_rate': 0.2,  # tried 0.5 and no dropout
#     'bn_momentum': 0.9,  # tried default of 0.99
#     'fc1_num_neurons': 1024,
#     'fc2_num_neurons': 512,
#     'fc3_num_neurons': 256,
#     'hidden_activation': 'relu',
#     'output_activation': 'sigmoid',  # tried softmax
#     'loss_function': 'binary_crossentropy',
#     'metrics': ['accuracy'],
# }

# tried sgd
# tried splitting data into train-val-test instead of using test set as val set

defaults = {
    'epochs': 20,
    'batch_size': 128,
    'fc1_num_neurons': 1024,
    'fc2_num_neurons': 512,
    'fc3_num_neurons': 256,
    'seed': 7,
    'learning_rate': 1e-3,
    'optimizer': 'adam',
    'hidden_activation': 'relu',
    'output_activation': 'sigmoid',
    'loss_function': 'binary_crossentropy',
    'metrics': ['accuracy'],
}

run = wandb.init(
    name='mobilenet_v2',
    project='cz4042',
    config=defaults,
)

config = wandb.config

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)


Problem at: <ipython-input-6-f1909b1a6d70> 42 <module>


UsageError: ignored

Load data

In [ ]:
project_dir_path = '/content/drive/My Drive/Adience/'
data_file_path = project_dir_path + 'aligned_gender.csv'


dataframe = pd.read_csv(data_file_path, sep=',', names=['image_path', 'gender'])
dataframe['image_path'] = project_dir_path + dataframe['image_path'].astype(str)

In [ ]:
# confirm dataframe is ok
dataframe.head()

In [ ]:
# check number of data in dataframe
dataframe.shape

(12194, 2)

In [ ]:
# check percentage of gender in data
dataframe['gender'].value_counts() / dataframe.shape[0] * 100

0    52.665245
1    47.334755
Name: gender, dtype: float64

Not very imbalanced; good

Split data into train-validation-test

In [ ]:
train_df, test_df = train_test_split(
    dataframe,
    test_size=0.2,
    random_state=config.seed,
)

In [ ]:
print(train_df.shape)
print(test_df.shape)

(9755, 2)
(2439, 2)


In [ ]:
train_df['gender'].value_counts() / train_df.shape[0] * 100

0    52.598667
1    47.401333
Name: gender, dtype: float64

In [ ]:
# Load images into keras image generator 
datagen_train = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)
datagen_test = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)

train_generator = datagen_train.flow_from_dataframe(
    dataframe=train_df,
    # directory=project_dir_path,
    x_col='image_path',
    y_col='gender',
    batch_size=config.batch_size,
    seed=config.seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

test_generator = datagen_test.flow_from_dataframe(
    dataframe=test_df,
    # directory=project_dir_path,
    x_col='image_path',
    y_col='gender',
    batch_size=config.batch_size,
    seed=config.seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

In [ ]:
mobile_net_v2 = tf.keras.applications.MobileNetV2(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    input_shape=(224,224,3),
)
mobile_net_v2.trainable = False

fc1 = tf.keras.layers.Dense(
    config.fc1_num_neurons,
    activation=config.hidden_activation,
)

fc2 = tf.keras.layers.Dense(
    config.fc2_num_neurons,
    activation=config.hidden_activation,
)

fc3 = tf.keras.layers.Dense(
    config.fc2_num_neurons,
    activation=config.hidden_activation,
)

model = tf.keras.models.Sequential([
    mobile_net_v2,
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    fc1,
    tf.keras.layers.BatchNormalization(),
    fc2,
    tf.keras.layers.BatchNormalization(),
    fc3,
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation=config.output_activation),
])

model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 1280)              2257984   
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1280)              5120      
_________________________________________________________________
dense (Dense)                (None, 1024)              1311744   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
___________________________________________________________

In [ ]:
# Compile model 
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
    loss=config.loss_function,
    metrics=config.metrics,
)

Define callbacks for logging and saving weights while training.

In [ ]:
log_callback = wandb.keras.WandbCallback()

checkpoint_dir_path = project_dir_path + 'checkpoints/'
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=(checkpoint_dir_path + 'cp-{epoch:02d}-{val_loss:.2f}.h5'),
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
)

In [ ]:
%%time
model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=config.epochs,
    callbacks=[log_callback, cp_callback],
)
model.save_weights('baseline_mobilenetv2_model.h5') 

Epoch 1/20
77/77 [==============================] - 2933s 38s/step - loss: 0.4382 - accuracy: 0.8194 - val_loss: 0.3473 - val_accuracy: 0.8479
Epoch 2/20
77/77 [==============================] - 132s 2s/step - loss: 0.1537 - accuracy: 0.9444 - val_loss: 0.3307 - val_accuracy: 0.8577
Epoch 3/20
77/77 [==============================] - 130s 2s/step - loss: 0.0762 - accuracy: 0.9730 - val_loss: 0.3162 - val_accuracy: 0.8754
Epoch 4/20
77/77 [==============================] - 130s 2s/step - loss: 0.0525 - accuracy: 0.9820 - val_loss: 0.3591 - val_accuracy: 0.8766
Epoch 5/20
77/77 [==============================] - 129s 2s/step - loss: 0.0445 - accuracy: 0.9858 - val_loss: 0.3826 - val_accuracy: 0.8872
Epoch 6/20
77/77 [==============================] - 130s 2s/step - loss: 0.0450 - accuracy: 0.9829 - val_loss: 0.4421 - val_accuracy: 0.8717
Epoch 7/20
77/77 [==============================] - 129s 2s/step - loss: 0.0597 - accuracy: 0.9788 - val_loss: 0.4227 - val_accuracy: 0.8758
Epoch 8/20


Evaluate on test set

In [ ]:
%%time
results = model.evaluate(
    test_generator,
    batch_size=config.batch_size,
    callbacks=[log_callback],
)

20/20 [==============================] - 25s 1s/step - loss: 0.5616 - accuracy: 0.8852
CPU times: user 25.4 s, sys: 905 ms, total: 26.4 s
Wall time: 27.7 s


Let Weights and Biases know that this run is complete.

In [ ]:
run.finish()

epoch,19
loss,0.00707
accuracy,0.99744
val_loss,0.56159
val_accuracy,0.8852
_step,19
_runtime,5492
_timestamp,1604402912
best_val_loss,0.31618
best_epoch,2


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▃▂▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁
accuracy,▁▆▇▇█▇▇█████▇███████
val_loss,▂▁▁▂▃▄▃▄▅▆▆▆▅█▅▅▆▅▅▇
val_accuracy,▁▃▅▆▇▅▅▆▇█▇▆▇▅▇█▇██▇
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_runtime,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_timestamp,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
